# 책정보 수집
item_id를 사용하여 isbn, 카테고리, page 정보 수집
- 사용파일:A_information_etc_utf8.csv
- 생성파일: A_information_utf8.csv

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
from selenium import webdriver
import time
import csv
import pandas as pd

In [5]:
# CSV 파일 경로
# 알라딘-총평점-v1.0 작업 이후 A_information_etc 파일로 작업필요
file_path = 'A_information_etc_utf8.csv'

# pandas를 사용하여 CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 'item_id' 열의 값들을 중복 없이 리스트로 변환
item_id_list = df['item_id'].unique().tolist()

In [8]:
information_list = []
information_html_list = []

In [9]:
wd = webdriver.Chrome()
count = 1
for ItemId in item_id_list:
    isbn = categories = page = None
    try:
        #알라딘상세페이지
        aladin_url =  f'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={ItemId}'
        wd.get(aladin_url)
        # 브라우져가 로딩이 완료될때까지 기다려야한다.
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 3초동안 기다림
        html = wd.page_source
        soup = BeautifulSoup(html,'html.parser')

        # print(soup.prettify())
        
        #메타태그속 isbn추가
        meta_tag = soup.find('meta', property='books:isbn')
        if meta_tag:
            isbn = meta_tag['content']
        else:
            isbn = None
        
        category_element = soup.find('ul', id='ulCategory')
        categories = category_element.text if category_element else "Category not found"
        
       #페이지
        li_page = soup.select('#Ere_prod_allwrap > div.Ere_prod_middlewrap > div:nth-child(1) > div.Ere_prod_mconts_R > div.conts_info_list1 > ul > li:nth-child(1)' )
        if li_page[0].get_text(strip=True)[:-1] in ["양장", "양장본", "보드북"]:
            li_page_re = soup.select('#Ere_prod_allwrap > div.Ere_prod_middlewrap > div:nth-child(1) > div.Ere_prod_mconts_R > div.conts_info_list1 > ul > li:nth-child(2)' )
            page = int(li_page_re[0].get_text(strip=True)[:-1]) 
        else:
            page = int(li_page[0].get_text(strip=True)[:-1])
        
        information_dict = {
            'item_id': ItemId,
            'isbn': isbn,
            'categories': categories,
            'page': page
        }
        print(count, time.strftime('%Y.%m.%d - %H:%M:%S'),information_dict)
        count += 1
        
        # 딕셔너리를 리스트에 추가
        information_list.append(information_dict)
        information_html = {
                'item_id': ItemId,
                'html': soup
            }
        information_html_list.append(information_html)
    except Exception as e:
        information_dict = {
            'item_id': ItemId,
            'isbn': isbn,
            'categories': categories,
            'page': page
        } 
        information_list.append(information_dict)
        information_html = {
                'item_id': ItemId,
                'html': soup
            }
        information_html_list.append(information_html)
        continue



fields = list(information_list[0].keys())
# 'encoding' 인자를 'utf-8'로 설정
with open('information_list_2019.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(information_list)

1 2024.03.15 - 16:09:13 {'ItemId': '93577688', 'isbn': '9788965422006', 'categories': '\n국내도서\xa0>\xa0외국어\xa0>\xa0토익\xa0>\xa0Listening 접기국내도서\xa0>\xa0외국어\xa0>\xa0토익\xa0>\xa0종합 접기국내도서\xa0>\xa0외국어\xa0>\xa0토익\xa0>\xa0Reading 접기\n', 'page': 596}
2 2024.03.15 - 16:09:17 {'ItemId': '170482558', 'isbn': '9788932473901', 'categories': '\n국내도서\xa0>\xa0과학\xa0>\xa0생명과학\xa0>\xa0생명과학 접기국내도서\xa0>\xa0과학\xa0>\xa0생명과학\xa0>\xa0생물학 접기국내도서\xa0>\xa0과학\xa0>\xa0생명과학\xa0>\xa0진화론 접기국내도서\xa0>\xa0대학교재/전문서적\xa0>\xa0자연과학계열\xa0>\xa0과학일반 접기\n', 'page': 632}
3 2024.03.15 - 16:09:20 {'ItemId': '179428940', 'isbn': '9791130620459', 'categories': '\n국내도서\xa0>\xa0인문학\xa0>\xa0철학 일반\xa0>\xa0교양 철학 접기\n', 'page': 336}
4 2024.03.15 - 16:09:27 {'ItemId': '125385284', 'isbn': '9788965422396', 'categories': '\n국내도서\xa0>\xa0외국어\xa0>\xa0토익\xa0>\xa0Listening 접기\n', 'page': 626}
5 2024.03.15 - 16:09:30 {'ItemId': '175330696', 'isbn': '9791188700288', 'categories': '\n국내도서\xa0>\xa0좋은부모\xa0>\xa0부모교육 접기국내도서\xa0>\xa0좋은부모\xa0>\xa0교육/학습\x

In [10]:
len(information_list)
# information_html_list

201

In [11]:
fields = list(information_list[0].keys())
# 'encoding' 인자를 'utf-8'로 설정
with open('A_information_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(information_list)

In [81]:
fields = list(information_html_list[0].keys())
# 'encoding' 인자를 'utf-8'로 설정
with open('information_html_list.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(information_html_list)